In [1]:
# import sys
# print(sys.version)
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError("GPU device not found")
# print("Found GPU AT:{}".format(device_name))

In [2]:
import keras
import tensorflow as tf
keras.__version__

Using TensorFlow backend.


'2.1.3'

In [3]:
from keras import backend as K

# tensorflow2.0设置
# config = tf.compat.v1.ConfigProto(allow_soft_placement=True)

# config.gpu_options.allow_growth = True
# tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

# tensorflow>2.0的设置
# keras的设置如下
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.4
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

K.tensorflow_backend._get_available_gpus()

[]

In [4]:
print(tf.__version__)

1.14.0


# 5.1 - Introduction to convnets

This notebook contains the code sample found in Chapter 5, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

First, let's take a practical look at a very simple convnet example. We will use our convnet to classify MNIST digits, a task that you've already been 
through in Chapter 2, using a densely-connected network (our test accuracy then was 97.8%). Even though our convnet will be very basic, its 
accuracy will still blow out of the water that of the densely-connected model from Chapter 2.

The 6 lines of code below show you what a basic convnet looks like. It's a stack of `Conv2D` and `MaxPooling2D` layers. We'll see in a 
minute what they do concretely.
Importantly, a convnet takes as input tensors of shape `(image_height, image_width, image_channels)` (not including the batch dimension). 
In our case, we will configure our convnet to process inputs of size `(28, 28, 1)`, which is the format of MNIST images. We do this via 
passing the argument `input_shape=(28, 28, 1)` to our first layer.

In [5]:
from keras import layers
from keras import models

model = models.Sequential()
'''
卷积层
第一层卷积层定义了32个滤镜，每个滤镜的大小是3*3，这样经过滤镜处理后的图片大小为：
    (28 - 3 + 1)/1(stride=1) = 0
    (width/height - filter + 1 + p)/s

最后经过滤镜得到的结果用relu进行非线性转换。
关于Conv2D方法的用法可以查看官方文档。
参数量计算为：
    第一层的卷积核为：3*3，每一个卷积核的参数为：3*3*上一层的通道数+biase项=3*3*1+1
    第一层的channel有32个，所以所需的参数为:每一个卷积核参数大小乘以当前层的channel = (3*3*1+1)*32=320
    从而第一层的总共参数大小为320个。
    
'''
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
'''
maxpooling : 池化层
    第一层池化层大小为(2,2)，这样经过池化后的图片大小为：(26/2,26/2)=(13,13)
    从而经过池化后的图片大小变为了(13,13,32):意思是有32个通道的(13,13)大小的图片
'''
model.add(layers.MaxPooling2D((2, 2)))
'''
一个滤镜所需要的参数量为：滤镜大小为3*3，上一层滤镜数为32，偏置项个数1。从而一个滤镜所需要
的参数量为3*3*32+1,现在该卷积层有64个滤镜，从而需要的总参数量为(3*3*32+1)*64

    图片(width,hegiht) = ((13-3+1)/1,(13-3+1)/1)=(11,11)
'''
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
'''
第二层池化层，
    图片(width,height) = (11/2,11/2) = (5,5) 向下取整
'''
model.add(layers.MaxPooling2D((2, 2)))
# 最后一层卷积需要的参数大小为：(3*3*64+1)*64
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [10]:
?layers.Conv2D

In [9]:
?layers.MaxPooling2D

Let's display the architecture of our convnet so far:

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


You can see above that the output of every `Conv2D` and `MaxPooling2D` layer is a 3D tensor of shape `(height, width, channels)`. The width 
and height dimensions tend to shrink as we go deeper in the network. The number of channels is controlled by the first argument passed to 
the `Conv2D` layers (e.g. 32 or 64).

The next step would be to feed our last output tensor (of shape `(3, 3, 64)`) into a densely-connected classifier network like those you are 
already familiar with: a stack of `Dense` layers. These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor. 
So first, we will have to flatten our 3D outputs to 1D, and then add a few `Dense` layers on top:

In [12]:
# 将图片摊平，变为一维
model.add(layers.Flatten())
# 该层是fully connect
# 参数计算为：（1*1*576+1）*64=36928,可以认为全连接层的卷积核大小为1*1。
model.add(layers.Dense(64, activation='relu'))
# 该层也是fully connect,最后输出
# 参数计算为：(1*1*64+1)*10=650
model.add(layers.Dense(10, activation='softmax'))

We are going to do 10-way classification, so we use a final layer with 10 outputs and a softmax activation. Now here's what our network 
looks like:

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

As you can see, our `(3, 3, 64)` outputs were flattened into vectors of shape `(576,)`, before going through two `Dense` layers.

Now, let's train our convnet on the MNIST digits. We will reuse a lot of the code we have already covered in the MNIST example from Chapter 
2.

In [14]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 图片需要变为4维张量
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# one-hot encoding to_categorical相当于one-hot encoding.
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [16]:
print(train_labels.shape)
print(train_labels[0])

(60000, 10)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [17]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/5
60000/60000 [==============================] - 20s 339us/step - loss: 0.1802 - acc: 0.9432
Epoch 2/5
60000/60000 [==============================] - 20s 329us/step - loss: 0.0466 - acc: 0.9853
Epoch 3/5
60000/60000 [==============================] - 20s 334us/step - loss: 0.0315 - acc: 0.98985s - loss: 0.0320 - - ETA:  - ETA: 2s - - ETA: 1s - loss: 0
Epoch 4/5
60000/60000 [==============================] - 20s 326us/step - loss: 0.0234 - acc: 0.9924
Epoch 5/5
60000/60000 [==============================] - 19s 325us/step - loss: 0.0179 - acc: 0.9941


Let's evaluate the model on the test data:

In [18]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 84us/step


In [19]:
test_acc

0.9914

While our densely-connected network from Chapter 2 had a test accuracy of 97.8%, our basic convnet has a test accuracy of 99.3%: we 
decreased our error rate by 68% (relative). Not bad! 

## The convolution operation

+ The fundamental difference between a densely connected layer and a convolution layer is this:Dense layers learn global patterns in their input feature space(for example,for a MNIST digit,patterns involving all pixels),whereas convolution layers learn local patterns.

+ This key characteristic gives convnets two interesting properties.
>+ The patterns they learn are translation invariant.After learning a certain pattern in the low-right corner of a picture,a convnet can recognize it anywhere:for example,in the upper-left corner.A densely connected network would have to learn the pattern anew if it appeared at a new location.This makes convnets data efficient when processing images(because the visual world is fundamentally translation invariant):they need fewer training samples to learn representations that have generalization power.
>+ They can learn spatial hierarchies of patterns.A first convolution layer will learn small local patterns such as edges,a second convolution layer will learn larger patterns made of the features of the first layers,and so on.This allows convnets to efficiently learn increasingly complex and abstract visual concepts(because the visual world is fundamentally spatially hierachical).

+ Convolutions operate over 3D tensors,called feature maps,with two spatial axes(height and width) as well as a depth axis(also called the channels axis).
+ The convolution operation extracts patches from its input feaure map and applies the same transformation to all of these patches producing an ouput feature amp.This ooutput feature map is still a 3D tensor:It has a width and a height.
+ Its depth can be arbitrary,because the output depth is a parameter of the layer,and the current channels in that depth axis no longer stand for specific colors as in RGB input;rather,they stand for filters.
+ In the MNIST examples,the first convolution layer takes a feature map of size(28,28,1) and outputs a feature map of size(26,26,32);it computes 32 filters over its input.Each of these 32 output channels contains a 26*26 grid of values,which is a response map f the filter over the input,indicating the response of that filter pattern at different locations in the input.

+ Convolutions are defined by two key parameters:
>+ Size of the patches extracted from the inputs--These are typically 3*3 or 5*5.In the example,they were 3*3,which is a common choice.
>+ Depth of the ouotput feature map--The number of filters computed by the convolution.The example started with a depth of 32 and ended with a depth of 64.


+ Note that the output width and height may differ from the input width and hegiht.They may differ for two reasons:
>+ Border effects,which can be countered by padding the input feature map.In Conv2D layers,padding is configurable via the padding argument,which takes two values:"valid",which means no padding(only valid window locations will be used);and "same",which means "pad in such a way as to have an output with the same width and height as the input."The padding argument defaults to "valid".
>+ The use of stride.The distance between two successive windows is a paramter of the convolution,called its stride,which defaults to 1.It's possible to have strided convolutions:convolutions with a stride higher than 1.Using stride 2 means the width and height of the feature map are downsampled by a factor 2((in addition to any changes induced by border effects).Strided convolutions are rarely used in practice.

## The max-pooling operation
+ The role of max pooling:to aggressively downsample feature maps,much like strided convolutions.
+ It's conceptually similar to convolutin,except thath instead of transforming local patches via a learned linear transformation,they're transformed via a hardcoded max tensor operation.
+ A big difference from convolutions is that max pooling is usually done with 2*2 windows and stride 2,in order to downsample the feature maps by a factor of 2.On the other hand,convolutions is typically done with 3*3 windows and no stride(stride 1).
+ Why downsample feature maps this way?Why not remove the max-pooling layers and keep fairly large feature maps all the way up?Let's look at this option.The convolutional base of the model would then look like this:

In [ ]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32,(3,3),activation='relu',
                                    input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64,(3,3),activation='relu'))
model_no_max_pool.add(layers.Conv2D(64,(3,3),activation='relu'))

In [ ]:
model_no_max_pool.summary()

+ What's wrong with this setup?Two things:
>+ It isn't conductive to learning a spatial hierachy of features.The 3*3 windows in the third layer will only contain information coming from 7*7 windows in the initial input.The hihg-level patterns learned bby the convnet will still be very small with regard to the initial input,which may not be ennough to learn to classify digits.We need the features from the last convolution layer to contain information about the totality of the input.
>+ The final feature map has 22*22*64=30976 total coefficients per sample.This is huge.If you were to flattern it to stick a Dense layer of size 512 on top,that layer would have 15.8 million parameters.

+ In short,the reason to use downsampling is to reduce the number of eature-map coefficients to process,as well as to induce spatial-filter hierarchies by making successive convolution layers look at increasingly large windows(in terms of the fraction of the original input they cover).
+ You can use average pooling instead of max pooling,where each local input patch is transfomed by taking the average value of each channel over the patch,rather than the max.But max pooling tends to work better than these alternative solutions.
+ In a nutshell,the reason is that features tend to encode the spatial presence of some pattern or concept over the different tiles of the feature map(hence,them feature map),and it's more information to look at the maximal presence of different features than at their average presence.
+ The most reasonable subsampling strategy is to first produce dense maps of features(via unstrided convolutions) and then look at the maximal activation of the features over small patches,rather than looking at sparser windows of the inputs(via strided convolutions) or averaging input ptaches.